In [1]:
import pandas as pd
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
from peft import LoraConfig, get_peft_model
from torch import nn
from photutils import CircularAnnulus, EllipticalAperture
from photutils.aperture import aperture_photometry
from torch.utils.data.sampler import Sampler
from collections import defaultdict
import random
from astropy.stats import sigma_clip


/tmp/ipykernel_67/4008959035.py:8: DeprecationWarning: `photutils.CircularAnnulus` is a deprecated alias for `photutils.aperture.CircularAnnulus` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAnnulus` to silence this warning.
  from photutils import CircularAnnulus, EllipticalAperture
/tmp/ipykernel_67/4008959035.py:8: DeprecationWarning: `photutils.EllipticalAperture` is a deprecated alias for `photutils.aperture.EllipticalAperture` and will be removed in the future. Instead, please use `from photutils.aperture import EllipticalAperture` to silence this warning.
  from photutils import CircularAnnulus, EllipticalAperture


In [2]:
#load the data
df_train = pd.read_csv('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/df_train.csv')

stamp_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/stamp_train.npy')
mask_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_train.npy')

In [3]:
#The function to calculate the flux The original version
#Wrote by Jiefeng Chen

cutout_size = 48
def background_annulus_jiefeng(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')

    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    background_annulus = np.ma.mean(clip_annulus_array)
    #we use median here, in the dataset they use mean
    #background_annulus = np.ma.median(clip_annulus_array)
    return background_annulus

def flux_elliptical_jiefeng(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    image_shape = (cutout_size*2,cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    image_good = image * (1 - xmask)
    
    raw_flux = np.sum(image_good * mask_image_photutils_fractional)#calculate by myself

    background = background_annulus_jiefeng(image, mask, aperture_x, aperture_y)
    gal_flux = raw_flux - source_aperture.area * background
    
    return gal_flux

In [4]:
cutout_size = 48

def background_annulus_jiefeng(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    if hasattr(data, 'shape') and len(data.shape) == 4:  # [batch, channel, height, width]
        return background_annulus_jiefeng_batch(data, mask, aperture_x, aperture_y, r_in, r_out)
    
    if hasattr(data, 'is_cuda'):
        device = data.device
        data_np = data.cpu().detach().numpy()
        mask_np = mask.cpu().detach().numpy()
        aperture_x_np = aperture_x.cpu().detach().numpy()
        aperture_y_np = aperture_y.cpu().detach().numpy()
    else:
        data_np = data
        mask_np = mask
        aperture_x_np = aperture_x
        aperture_y_np = aperture_y
    
    masked_data = np.ma.array(data=data_np, mask=mask_np != 0)
    masked_data = masked_data.filled(fill_value=0)
    masked_data = masked_data.squeeze()
    
    center = (aperture_x_np, aperture_y_np)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')
    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    background_annulus = np.ma.mean(clip_annulus_array)
    
    if hasattr(data, 'is_cuda'):
        return torch.tensor(background_annulus, dtype=data.dtype, device=device)
    else:
        return background_annulus

def flux_elliptical_jiefeng(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    if hasattr(image, 'shape') and len(image.shape) == 4:  # [batch, channel, height, width]
        return flux_elliptical_jiefeng_batch(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b)
    
    if hasattr(image, 'is_cuda'):
        device = image.device
        image_np = image.cpu().detach().numpy()
        mask_np = mask.cpu().detach().numpy()
        aperture_x_np = aperture_x.cpu().detach().numpy()
        aperture_y_np = aperture_y.cpu().detach().numpy()
        aperture_theta_np = aperture_theta.cpu().detach().numpy()
        aperture_a_np = aperture_a.cpu().detach().numpy()
        aperture_b_np = aperture_b.cpu().detach().numpy()
    else:
        image_np = image
        mask_np = mask
        aperture_x_np = aperture_x
        aperture_y_np = aperture_y
        aperture_theta_np = aperture_theta
        aperture_a_np = aperture_a
        aperture_b_np = aperture_b

    image_shape = (cutout_size*2, cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta_np * np.pi / 180.
    a = aperture_a_np / PIXEL_SCALE
    b = aperture_b_np / PIXEL_SCALE

    center = (aperture_x_np, aperture_y_np)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    xmask = mask_np != 0
    image_good = image_np * (1 - xmask)
    raw_flux = np.sum(image_good * mask_image_photutils_fractional)

    background = background_annulus_jiefeng(image_np, mask_np, aperture_x_np, aperture_y_np)
    gal_flux = raw_flux - source_aperture.area * background
    
    if hasattr(image, 'is_cuda'):
        return torch.tensor(gal_flux, dtype=image.dtype, device=device)
    else:
        return gal_flux

In [54]:
def background_annulus_jiefeng_batch(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    
    if len(data.shape) == 4:  # [batch, channel, height, width]
        batch_size = data.shape[0]
        device = data.device
        dtype = data.dtype
        
        data_np = data.cpu().detach().numpy()
        mask_np = mask.cpu().detach().numpy()
        aperture_x_np = aperture_x.cpu().detach().numpy()
        aperture_y_np = aperture_y.cpu().detach().numpy()
        
        if data_np.shape[1] > 1:
            data_np = data_np[:, 0, :, :]  # 取第一个通道
            mask_np = mask_np[:, 0, :, :]  # 取第一个通道
        
        backgrounds = []
        for i in range(batch_size):
            bg = background_annulus_jiefeng(
                data_np[i], mask_np[i],
                aperture_x_np[i], aperture_y_np[i],
                r_in, r_out
            )
            backgrounds.append(bg)
        
        return torch.tensor(backgrounds, dtype=dtype, device=device)
    else:
        return background_annulus_jiefeng(data, mask, aperture_x, aperture_y, r_in, r_out)

def flux_elliptical_jiefeng_batch(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    if len(image.shape) == 4:  # [batch, channel, height, width]
        batch_size = image.shape[0]
        device = image.device
        dtype = image.dtype
        
        image_np = image.cpu().detach().numpy()
        mask_np = mask.cpu().detach().numpy()
        aperture_x_np = aperture_x.cpu().detach().numpy()
        aperture_y_np = aperture_y.cpu().detach().numpy()
        aperture_theta_np = aperture_theta.cpu().detach().numpy()
        aperture_a_np = aperture_a.cpu().detach().numpy()
        aperture_b_np = aperture_b.cpu().detach().numpy()
        
        if image_np.shape[1] > 1:
            image_np = image_np[:, 0, :, :]  # 取第一个通道
            mask_np = mask_np[:, 0, :, :]    # 取第一个通道
        
        fluxes = []
        for i in range(batch_size):
            flux = flux_elliptical_jiefeng(
                image_np[i], mask_np[i],
                aperture_x_np[i], aperture_y_np[i],
                aperture_theta_np[i], aperture_a_np[i], aperture_b_np[i]
            )
            fluxes.append(flux)
        
        return torch.tensor(fluxes, dtype=dtype, device=device)
    else:
        return flux_elliptical_jiefeng(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b)

In [35]:
#The class to arrange the lines of the dataframe
#Can see the below comments

class PairedBatchSampler(Sampler):
    """
    choose P ref_ids, and choose 2 from each ref_id.
    
    P = 8, K = 2 , batch_size = 16
    batch_indices = [idx_A1, idx_G1, ... idx_F1,   idx_A2, idx_G2, ... idx_F2]
                  
    - ref_ids (list)
    - P (int): number of the pair
    """
    def __init__(self, ref_ids, P):
        super(PairedBatchSampler, self).__init__()
        
        if P <= 0:
            raise ValueError("P must be > 0")

        self.P = P
        self.K_is_fixed_at = 2
        self.batch_size = P * self.K_is_fixed_at
        
        print("constructing PairedBatchSampler ...")
        grouped_indices = defaultdict(list)
        for i, ref_id in enumerate(ref_ids):
            grouped_indices[ref_id].append(i)
        
        print(f"creating 'chunks' (size K={self.K_is_fixed_at})...")
        self.all_chunks = []
        for ref_id, indices in grouped_indices.items():
            if len(indices) >= self.K_is_fixed_at: #Acutally I have already selected the ref_ids
                
                random.shuffle(indices) #make it random
                
                # divide the ref_id. e.g. floor(13 / 4) = 3. we build 3 chunks。
                num_chunks_for_this_id = len(indices) // self.K_is_fixed_at
                
                for i in range(num_chunks_for_this_id):
                    chunk = indices[i * self.K_is_fixed_at : (i + 1) * self.K_is_fixed_at]
                    self.all_chunks.append(chunk)
        
        print(f"Already created {len(self.all_chunks)} 'K-chunks'.")
        
        if len(self.all_chunks) < P:
            raise ValueError(f"ref_ids are less than P={P} 'K-chunks'. Use a smaller K or P")
            
        self.num_batches = len(self.all_chunks) // P

    def __iter__(self):
        
        random.shuffle(self.all_chunks)
        
        for i in range(self.num_batches):
            batch_part1_indices = []
            batch_part2_indices = []
            
            p_chunks = self.all_chunks[i * self.P : (i + 1) * self.P]
            
            for chunk in p_chunks:
                
                batch_part1_indices.append(chunk[0])
                batch_part2_indices.append(chunk[1])
            
            final_batch_indices = batch_part1_indices + batch_part2_indices
            
            # P + P = 16
            yield final_batch_indices

    def __len__(self):
        return self.num_batches

In [36]:
#Select and arrange the dataframe
P = 8
sampler = PairedBatchSampler(df_train['ref_id'], P=P)
print("\n getting index from the sampler...")
list_of_all_batches = list(sampler)

shuffled_indices = [index for batch in list_of_all_batches for index in batch]
print(f"Have created {len(shuffled_indices)} lines")

shuffled_df_train = df_train.iloc[shuffled_indices]
shuffled_df_train = shuffled_df_train.reset_index(drop=True)

#Select and arrange the stamps and the masks, to let them correspond to the dataframe
original_indices = shuffled_df_train['Unnamed: 0'].values
shuffled_stamp_train = stamp_train[original_indices]
shuffled_mask_train = mask_train[original_indices]

#the stamps and the masks have the same lines.
print(shuffled_stamp_train.shape)
print(shuffled_mask_train.shape)

constructing PairedBatchSampler ...
creating 'chunks' (size K=2)...
Already created 8914 'K-chunks'.

 getting index from the sampler...
Have created 17824 lines
(17824, 96, 96)
(17824, 96, 96)


In [37]:
#Can have a look
#focus on the first 8 and the second 8 lines, they have the same ref_ids, but different zp
shuffled_df_train.head(16)

,Unnamed: 0,ref_id,I_auto,zp,aperture_x,aperture_y,aperture_theta,aperture_a,aperture_b,path
0,2356,79546,21.720,4.305321,186.30272,1331.65650,26.3742,1.755914,1.433954,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
1,7301,103184,21.822,4.457064,162.85896,288.20782,-20.8183,1.084244,1.027059,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
2,9372,53722,21.732,4.628127,126.61045,3046.24220,13.1246,1.120617,1.116266,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
3,11460,104987,20.212,4.439700,1487.13380,3975.30600,-30.6014,0.982206,0.901591,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
4,9792,39432,21.542,4.433870,1277.76370,3765.93550,-10.0248,1.582380,1.525308,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
5,11011,25885,21.398,4.088120,1694.33810,286.77792,-44.5391,1.143545,0.831600,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
6,14519,28633,19.698,4.183516,679.72080,156.59879,-66.9148,1.646935,1.601665,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
7,10044,40347,21.152,4.357286,104.92251,257.97690,8.4777,1.679332,1.335814,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
8,2354,79546,21.720,4.449712,169.12453,1390.26270,26.3742,1.684446,1.351282,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
9,7300,103184,21.822,4.390844,89.90463,289.54350,-20.8183,0.951919,0.891747,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...


In [39]:
#To convert them into tensor
tensor_stamp_train = torch.from_numpy(shuffled_stamp_train).float()
tensor_mask_train = torch.from_numpy(shuffled_mask_train).float()

tensor_stamp_train = tensor_stamp_train.unsqueeze(1)
tensor_mask_train = tensor_mask_train.unsqueeze(1)

#We just need 'ref_id', 'zp', 'aperture_x', 'aperture_y', 'aperture_theta', 'aperture_a', 'aperture_b'
features_df_train = shuffled_df_train[['ref_id', 'zp', 'aperture_x', 'aperture_y', 'aperture_theta', 'aperture_a', 'aperture_b']].values

aperture_x_original = features_df_train[:, 2]  # 第三列
aperture_y_original = features_df_train[:, 3]  # 第四列

aperture_x_new = cutout_size + aperture_x_original - np.floor(aperture_x_original)
aperture_y_new = cutout_size + aperture_y_original - np.floor(aperture_y_original)

features_df_train[:, 2] = aperture_x_new
features_df_train[:, 3] = aperture_y_new

features_df_train_tensor = torch.FloatTensor(features_df_train)

In [40]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(tensor_stamp_train, tensor_mask_train, features_df_train_tensor)
traindataloader = DataLoader(dataset, batch_size=16, shuffle=False)
#I set shuffle=False here, because I have already do the shuffle before

In [58]:
#I have already simplified the loss functions
def PairedDifferenceLoss(outputs, masks, zps, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    
    batch_size = outputs.shape[0]

    gal_flux_output = flux_elliptical_jiefeng(outputs, masks, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)

    gal_output_calibrated = gal_flux_output * zps
    #divide into 2
    half_B = batch_size // 2
        
    outputs_1 = gal_output_calibrated[0:half_B]
    outputs_2 = gal_output_calibrated[half_B:]
    loss = ((outputs_1 - outputs_2)**2).mean() 
    
    return loss

def loss_unbias(labels, outputs, masks, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    batch_size = outputs.shape[0]
    
    gal_flux_label = flux_elliptical_jiefeng(labels, masks, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)
    gal_flux_output = flux_elliptical_jiefeng(outputs, masks, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)

    loss = (gal_flux_label - gal_flux_output).abs().sum()
    
    return loss

In [87]:
#load the pn2v model
import os
os.chdir('/data/aai/scratch/jchan/denoise/PAUS/dinggetest/simulation/pn2v/src/pn2v')
from core import prediction
from core import utils
from unet import UNet

device=utils.getDevice()

CUDA available? True


In [88]:
path='/data/aai/scratch/jchan/denoise/PAUS/dinggetest/simulation/model saved/'
model=torch.load(path+"/best_conv_N2V_PAUdm.net")

/tmp/ipykernel_66/4130286196.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load(path+"/best_conv_N2V_PAUdm.net")


In [89]:
for param in model.parameters():
    param.requires_grad = False

In [90]:
#print(model)
target_list = [
    'conv_final',
    'down_convs.0.conv1',
    'down_convs.0.conv2',
    'down_convs.1.conv1',
    'down_convs.1.conv2',
    'down_convs.2.conv1',
    'down_convs.2.conv2',
    'up_convs.0.conv1',
    'up_convs.0.conv2',
    'up_convs.1.conv1',
    'up_convs.1.conv2'
]

In [91]:
#I add some Chinese comments here to explain some parameters...

config = LoraConfig(
    r=8,  #它控制了 LoRA 模块的“大小”或“复杂度”。r 越大，LoRA 模块的可训练参数就越多，理论上能学习更复杂的调整，但也会占用更多显存。r=8 或 16 是一个非常常见且高效的选择。
    lora_alpha=16, #LoRA 的输出会乘以一个缩放比例 alpha/r。这就像一个特殊的“学习率”或“平衡旋钮”。一个常见的经验法则是将 lora_alpha 设置为 r 的两倍（比如 r=8, alpha=16），这有助于稳定训练。
    target_modules=target_list,
    lora_dropout=0.1,#在 LoRA 模块中添加一个 Dropout 层，用于防止过拟合，这是一个标准的正则化技术。To prevent overfitting
)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 84,560 || all params: 1,761,938 || trainable%: 4.7993


In [62]:
num_epochs = 1
lora_model.train()

optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    for tensor_stamp_train, tensor_mask_train, features_df_train_tensor in traindataloader:
        
        inputs = tensor_stamp_train.to(device)
        masks = tensor_mask_train.to(device) 
        features = features_df_train_tensor.to(device)
        optimizer.zero_grad()
        
        outputs = lora_model(inputs) 
        #'ref_id', 'zp', 'aperture_x', 'aperture_y', 'aperture_theta', 'aperture_a', 'aperture_b'
        loss1 = loss_unbias(inputs, outputs, masks, features[:,2],features[:,3],features[:,4],features[:,5],features[:,6])
        loss2 = PairedDifferenceLoss(outputs, masks, features[:,1],features[:,2],features[:,3],features[:,4],features[:,5],features[:,6])
        loss1.requires_grad_(True)
        loss2.requires_grad_(True)
        
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

tensor(151.1360, device='cuda:0', grad_fn=<AddBackward0>)


ZeroDivisionError: division by zero

In [92]:
from tqdm import tqdm

num_epochs = 20
lora_model.train()
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-6)

for epoch in range(num_epochs):
    total_loss1 = 0.0
    total_loss2 = 0.0
    num_batches = 0
    
    progress_bar = tqdm(traindataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
    for tensor_stamp_train, tensor_mask_train, features_df_train_tensor in progress_bar:
        
        inputs = tensor_stamp_train.to(device)
        masks = tensor_mask_train.to(device) 
        features = features_df_train_tensor.to(device)
        optimizer.zero_grad()
        
        outputs = lora_model(inputs) 
        
        loss1 = loss_unbias(inputs, outputs, masks, features[:,2],features[:,3],features[:,4],features[:,5],features[:,6])
        loss2 = PairedDifferenceLoss(outputs, masks, features[:,1],features[:,2],features[:,3],features[:,4],features[:,5],features[:,6])
        loss1.requires_grad_(True)
        loss2.requires_grad_(True)
        
        loss = 0.01*loss1 + loss2
        
        loss.backward()
        optimizer.step()
        
        current_loss1 = loss1.item()
        current_loss2 = loss1.item()
        total_loss1 += current_loss1
        total_loss2 += current_loss2
        num_batches += 1
        
        progress_bar.set_postfix({
            'Loss': f'{current_loss1 + current_loss2:.6f}',
            'Avg Loss': f'{(current_loss1+current_loss2)/num_batches:.6f}'
        })
    
    avg_loss1 = total_loss1 / num_batches
    avg_loss2 = total_loss2 / num_batches
    
    print(f"\nEpoch {epoch+1} Completed - Average unbias Loss: {avg_loss1:.6f} Average df Loss {avg_loss2:.6f}")

Epoch 1/20: 100%|██████████| 1114/1114 [01:57<00:00,  9.52it/s, Loss=403.953674, Avg Loss=0.362616]



Epoch 1 Completed - Average unbias Loss: 169.178932 Average df Loss 169.178932


Epoch 2/20: 100%|██████████| 1114/1114 [01:57<00:00,  9.46it/s, Loss=403.953674, Avg Loss=0.362616]



Epoch 2 Completed - Average unbias Loss: 169.178932 Average df Loss 169.178932


Epoch 3/20: 100%|██████████| 1114/1114 [01:55<00:00,  9.63it/s, Loss=403.953674, Avg Loss=0.362616]


...


Epoch 18 Completed - Average unbias Loss: 169.178932 Average df Loss 169.178932


Epoch 19/20: 100%|██████████| 1114/1114 [02:00<00:00,  9.22it/s, Loss=403.953674, Avg Loss=0.362616]



Epoch 19 Completed - Average unbias Loss: 169.178932 Average df Loss 169.178932


Epoch 20/20: 100%|██████████| 1114/1114 [01:55<00:00,  9.65it/s, Loss=403.953674, Avg Loss=0.362616]


Epoch 20 Completed - Average unbias Loss: 169.178932 Average df Loss 169.178932
